# Model for evaporation of water and crystallization of dissolved salt

## Variables
- $\varphi_l, \varphi_c, \varphi_v$ : phase fields (liquid, crystal, vapor) with $\varphi_l+\varphi_c+\varphi_v=1$
- $s$ : salt concentration
- $\mu_l, \mu_c, \mu_v, \mu_s$ : chemical potentials
- $\kappa$ : Lagrange multiplier for constraint

## Energy
$$
\mathscr{E}(\varphi,s)=\int_\Omega \sum_{i\in\{l,c,v\}} \gamma_i\left[\frac{\varepsilon}{2}|\nabla\varphi_i|^2 + \frac{1}{\varepsilon}W(\varphi_i)\right] + s\ln s + (1-s)\ln(1-s) + \beta\varphi_c(s-s_{\mathrm{sat}}) + \lambda\varphi_v s \,\mathrm{d}x
$$

## Lagrangian
$$
\mathscr{L}(\varphi,s,\kappa) = \mathscr{E}(\varphi,s) + \int_\Omega \kappa(\varphi_l+\varphi_c+\varphi_v-1)\,\mathrm{d}x
$$

with double-well potential $W(\varphi)=18\varphi^2(1-\varphi)^2$ (penalized outside $[0,1]$).

## Dynamics
| Variable | Evolution |
|----------|-----------|
| $\varphi_l$ | $\partial_t\varphi_l = \nabla\cdot(m_l\nabla\mu_l) + h_e(\mu_v-\mu_l) + h_c(\mu_c-\mu_l)$ |
| $\varphi_c$ | $\partial_t\varphi_c = -h_c(\mu_c-\mu_l)$ |
| $\varphi_v$ | $\partial_t\varphi_v = \nabla\cdot(m_v\nabla\mu_v) - h_e(\mu_v-\mu_l)$ |
| $s$ | $\partial_t s = \nabla\cdot(m_s\nabla\mu_s)$ |

where $\mu_i = \delta\mathscr{E}/\delta\varphi_i$, $\mu_s = \delta\mathscr{E}/\delta s$, and mobilities $m_i$ depend on local phase composition.

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from dolfin import *
import logging
import numpy as np

logging.getLogger("FFC").setLevel(logging.ERROR)
logging.getLogger("UFL_LEGACY").setLevel(logging.ERROR)
logging.getLogger("UFL").setLevel(logging.ERROR)
set_log_level(LogLevel.ERROR)

# =============================================================================
# Parameter sets (compact format)
# =============================================================================
P_basic = dict(
    # Surface tensions: [γ_l, γ_c, γ_v]
    γ = [1/8, 1/8, 2.0],
    # Energy: [ε, s_sat, Λ]
    FP = [0.2,0.3,100],
    # Mobilities: [[m_ll,m_lv,m_lc], [m_vl,m_vv,m_vc], [m_sl,m_sv,m_sc]]
    m = [[1.0, 1.0, 0.01], [0.01, 1.0, 0.01], [0.01, 1.0, 0.01]],
    # Rates: [evaporation, crystallization]
    h = [1.0, 1.0],
    # Energy: [λ, β,c0]
    E = [10.0, -10.0, 0.01],
    # Geometry: [r0, L]
    G = [3.0, 4.0],
    # Simulation: [RADI, T]
    S = [True, 50],
    NAME = 'BASIC'
)

P1 = {**P_basic, 'E': [10,-10,0.01],'NAME': 'P1'} # Droplet evaporates and crystal remains
P2 = {**P_basic, 'E': [10,-10,0.10],'NAME': 'P2'} # Droplet evaporates a bit and crystal shell forms
P3 = {**P_basic, 'E': [1.,-1.,0.01],'NAME': 'P3'} # Droplet fully evaporates and no crystal remains
P4 = {**P_basic, 'E': [10,-1,0.1],'NAME': 'P4', 'S': [True, 100]} # Droplet evaporates a bit and stablizes with saline solution

P5 = {**P_basic, 'E': [1.,-1.,0.01],'NAME': 'P5', 'm': [[1e-3,1e-3,1e-3], [0.01, 1.0, 0.01], [0.01, 1.0, 0.01]]} # Droplet fully evaporates and no crystal remains

In [ ]:
# Main cell
from dolfin import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from ufl import ln
import numpy as np

PARS = P5

# Unpack
γ_l, γ_c, γ_v = PARS['γ']
(m_ll, m_lv, m_lc), (m_vl, m_vv, m_vc), (m_sl, m_sv, m_sc) = PARS['m']
he0, hc0 = PARS['h']
ε,s_sat,Λ = PARS['FP']
λ,β,c_0 = PARS['E']
r0, L = PARS['G']
RADI, T = PARS['S']
FNAME = PARS['NAME']

# FE definitions
mesh = IntervalMesh(128*3,0,L)
FE   = FiniteElement("P", mesh.ufl_cell(), 1)   # scalar element
Q    = FunctionSpace(mesh,MixedElement([FE,FE,FE,FE,FE,FE,FE,FE,FE]))# mixed space

if RADI:
  r = SpatialCoordinate(mesh)[0]**2
else:
  r = Constant(1.0)

def W(φ):
  W1 = conditional(lt(φ,0),Λ*φ**2,18*(φ*(1-φ))**2)
  W2 = conditional(gt(φ,1),Λ*(1-φ)**2,W1)
  return W2

# energy
def energy(q):
    φ_l,φ_c,φ_v,s,μ_l,μ_c,μ_v,μ_s,κ = split(q)
    
    E  = γ_l * ε/2 * inner(grad(φ_l),grad(φ_l))*r*dx
    E += γ_c * ε/2 * inner(grad(φ_c),grad(φ_c))*r*dx
    E += γ_v * ε/2 * inner(grad(φ_v),grad(φ_v))*r*dx
    E += γ_l/ε * W(φ_l)*r*dx
    E += γ_c/ε * W(φ_c)*r*dx
    E += γ_v/ε * W(φ_v)*r*dx
    # E += δ * inner(grad(s),grad(s))*dx

    E += (s*ln(s) + (1-s)*ln(1-s) + β*φ_c*(s-s_sat))*r*dx
    E += λ * φ_v * s *r*dx
    E += κ*(φ_l + φ_c + φ_v - 1)*r*dx
    # E += μ_inf * φ_v*r*dx
    return E


# single time step
def evolve(old_q, τ):
    # set up function spaces
    q,v = Function(Q),TestFunction(Q)
    φ_l,φ_c,φ_v,s,μ_l,μ_c,μ_v,μ_s,κ = split(q)
    vφ_l,vφ_c,vφ_v,vs,vμ_l,vμ_c,vμ_v,vμ_s,vκ = split(v)
    old_φ_l,old_φ_c,old_φ_v,old_s,old_μ_l,old_μ_c,old_μ_v,old_μ_s,old_κ = split(old_q)

    # define energy
    E = energy(q)
    m_s = m_sl*abs(old_φ_l) + m_sv*abs(old_φ_v) + m_sc*abs(old_φ_c)
    m_l = m_ll*abs(old_φ_l) + m_lv*abs(old_φ_v) + m_lc*abs(old_φ_c)
    m_v = m_vl*abs(old_φ_l) + m_vv*abs(old_φ_v) + m_vc*abs(old_φ_c)

    h_eva = he0*abs(old_φ_l)*abs(old_φ_v)
    h_cry = hc0*abs(old_φ_l) 

    # define weak form
    Res  = (μ_l*vφ_l + μ_c*vφ_c + μ_v*vφ_v + μ_s*vs)*r*dx - derivative(E, q, v)
    Res += m_l*τ*inner(grad(μ_l),grad(vμ_l))*r*dx + vμ_l*(φ_l-old_φ_l)*r*dx
    Res += vμ_c*(φ_c-old_φ_c)*r*dx
    Res += m_v*τ*inner(grad(μ_v),grad(vμ_v))*r*dx + vμ_v*(φ_v-old_φ_v)*r*dx
    
    Res += abs(old_s)*abs(1-old_s)*m_s*τ*inner(grad(μ_s),grad(vμ_s))*r*dx + vμ_s*(s-old_s)*r*dx

    Res +=  τ*(-h_eva*(μ_v-μ_l) - h_cry*(μ_c-μ_l) )*vμ_l*r*dx
    Res +=  τ*(                   h_cry*(μ_c-μ_l) )*vμ_c*r*dx
    Res +=  τ*( h_eva*(μ_v-μ_l)                   )*vμ_v*r*dx

    bc = []

    Jac     = derivative(Res, q)
    problem = NonlinearVariationalProblem(Res, q, bc, Jac)
    solver  = NonlinearVariationalSolver(problem)

    prm = solver.parameters
    prm['newton_solver']['error_on_nonconvergence'] = False
    prm['newton_solver']['report'] = False
    prm['newton_solver']['absolute_tolerance'] = 1e-5
    prm['newton_solver']['relative_tolerance'] = 1e-5

    q.assign(old_q)
    iterations, converged = solver.solve()
    
    dissi_ml = assemble(m_l*inner(grad(μ_l),grad(μ_l))*r*dx)
    dissi_mc = 0.0
    dissi_mv = assemble(m_v*inner(grad(μ_v),grad(μ_v))*r*dx)
    dissi_ms = assemble(abs(old_s)*abs(1-old_s)*m_s*inner(grad(μ_s),grad(μ_s))*r*dx)
    dissi_eva = assemble(h_eva*abs(μ_v-μ_l)**2*r*dx)
    dissi_cry = assemble(h_cry*abs(μ_c-μ_l)**2*r*dx)
    dissi = [dissi_ml,dissi_mc,dissi_mv,dissi_ms,dissi_eva,dissi_cry]
    e = assemble(E)

    return q,e,iterations,converged,dissi

# initial data
salt = 'c_0*exp(-λ * (1 - (1-0.5*(1+tanh(3*(x[0]-r0)/ε)))))'
iphil = '(1-0.5*(1+tanh(3*(x[0]-r0)/ε)))'
idata = Expression((iphil,"0",f'1-({iphil})',salt,"0","0","0","0","0"),degree=2,ε=ε,c_0=c_0,λ=λ,r0=r0)
old_q = interpolate(idata,Q)

old_q,e,it,conv,dissi = evolve(old_q,1e-4)
print('init:',it,conv)
times = []
energies = []
sols = []

list_dissi_ml = []
list_dissi_mc = []
list_dissi_mv = []
list_dissi_ms = []
list_dissi_eva = []
list_dissi_cry = []

# initial time stepping, later adaptive
t = 0
τ = 0.3e-2
n_steps = 4000
dt = Constant(τ)

for i in tqdm(range(n_steps)):
  dt.assign(τ)
  
  q,e,it,conv,dissi = evolve(old_q, dt)
  if conv:
    t += τ
    old_q.assign(q)
    if (i % 2 == 0):
      times.append(t)
      energies.append(e)
      sols.append(q)
      list_dissi_ml.append(dissi[0])
      list_dissi_mc.append(dissi[1])
      list_dissi_mv.append(dissi[2])
      list_dissi_ms.append(dissi[3])
      list_dissi_eva.append(dissi[4])
      list_dissi_cry.append(dissi[5])
    if it < 3:
      τ *= 1.1
    if it > 4:
      τ *= 0.95
  else:
    τ *= 0.75
  if t>T:
    break

In [ ]:
#
# 2D plotting of solutions
#
import numpy as np
import matplotlib.pyplot as plt

# choose a grid for evaluation
nx = 128*2
xgrid = np.linspace(0, L, nx)

# prepare array for space-time data
nt = len(sols)
S  = np.zeros((nt, nx))
FL = np.zeros((nt, nx))
FC = np.zeros((nt, nx))
FV = np.zeros((nt, nx))

# cycle over solution at different times
for i, q in enumerate(sols):
    φ_l,φ_c,φ_v,s,μ_l,μ_c,μ_v,μ_s,κ = q.split()
    S[i, :] = np.array([s(x) for x in xgrid])
    FL[i, :] = np.array([φ_l(x) for x in xgrid])
    FC[i, :] = np.array([φ_c(x) for x in xgrid])
    FV[i, :] = np.array([φ_v(x) for x in xgrid])

TT = np.array(times)
X  = xgrid
T0 = 0
DRANGE = 5
SHOW_ANALYTIC_PROFILE = False

if FNAME == 'P3':
    Tvis = 25
    SHOW_ANALYTIC_PROFILE = True
    s00 = 16.3
    ee = 0.3
    DRANGE = 4
if FNAME == 'P5':
    Tvis = 50
    SHOW_ANALYTIC_PROFILE = True
    s00 = 29
    ee = 0.5
    DRANGE = 0.4
if FNAME == 'P4':
    Tvis = 100
    DRANGE = 0.4
if FNAME == 'P2':
    Tvis = 50
    DRANGE = 2.0
if FNAME == 'P1':
    Tvis = 50
    DRANGE = 1
    
# Tvis = 100 # np.max(times)
xL = 0
xR = L
Ny = 2
Nx = 2

plt.figure(figsize=(9,7),constrained_layout=True)
plt.subplot(Ny,Nx,1)
plt.pcolormesh(X, TT, S, shading="auto",vmin=0,vmax=1)
plt.title("$s(t,r)$")
plt.xlabel("$r$")
plt.ylabel("$t$")
plt.ylim([T0,Tvis])
plt.xlim([xL,xR])
plt.colorbar()

plt.subplot(Ny,Nx,2)
plt.pcolormesh(X, TT, FL, shading="auto",vmin=0,vmax=1)
plt.title("$\\varphi_l(t,r)$")
plt.colorbar()
plt.xlabel("$r$")
plt.ylabel("$t$")

if SHOW_ANALYTIC_PROFILE:
    s = np.linspace(0,s00,100)
    plt.plot((np.abs(s00-s)**ee)*r0/s00**ee,s,'r:')

plt.xlim([xL,xR])
plt.ylim([T0,Tvis])

plt.subplot(Ny,Nx,3)
plt.pcolormesh(X, TT, FC, shading="auto",vmin=0,vmax=1)
plt.colorbar()
plt.title("$\\varphi_c(t,r)$")
plt.xlabel("$r$")
plt.ylabel("$t$")
plt.xlim([xL,xR])
plt.ylim([T0,Tvis])

plt.subplot(Ny,Nx,4)
plt.pcolormesh(X, TT, FV, shading="auto",vmin=0,vmax=1)
plt.colorbar()
plt.title("$\\varphi_v(t,r)$")
plt.xlabel("$r$")
plt.ylabel("$t$")
plt.xlim([xL,xR])
plt.ylim([T0,Tvis])
plt.savefig(f'Crystal_2x2_{FNAME}.png',dpi=300,bbox_inches='tight')

plt.figure(figsize=(4,3))
plt.plot(times,energies,label='energy')
plt.xlabel('$t$')
plt.ylabel('$\\mathscr{E}$')
plt.legend()
plt.xlim([T0,Tvis])
plt.grid(True)
plt.savefig(f'Energy_{FNAME}.pdf',dpi=300,bbox_inches='tight')

fig, ax1 = plt.subplots(figsize=(4,3))

# Linke Y-Achse: Energie
ax1.plot(times, energies, label='$\mathcal{E}$', color='C0',zorder=10)
ax1.set_xlabel('$t$')
ax1.set_ylabel('energy', color='k')
ax1.tick_params(axis='y', labelcolor='k')
ax1.set_xlim([T0, Tvis])
ax1.set_ylim([-2,22])
ax1.set_yticks(np.arange(0, 21, 5)) 

# Rechte Y-Achse: Dissipationen
ax2 = ax1.twinx()
ax2.plot(times, 0.5*np.array(list_dissi_eva),linestyle=':', label='$\mathcal{D}^*_{evap}$', color='C1',zorder=1)
ax2.plot(times, 0.5*np.array(list_dissi_cry),linestyle=':', label='$\mathcal{D}^*_{cryst}$', color='C2',zorder=1)
ax2.plot(times, 0.5*np.array(list_dissi_ml) ,linestyle=':', label='$\mathcal{D}^*_{m_\ell}$', color='C3',zorder=1)
#ax2.plot(times, list_dissi_mc ,linestyle=':', label='dissi mc', color='C4')
ax2.plot(times, 0.5*np.array(list_dissi_mv) ,linestyle=':', label='$\mathcal{D}^*_{m_v}$', color='C5',zorder=1)
ax2.plot(times, 0.5*np.array(list_dissi_ms) ,linestyle=':', label='$\mathcal{D}^*_{m_s}$', color='C6',zorder=1)
ax2.set_ylabel('dissipation', color='k')
ax2.tick_params(axis='y', labelcolor='k')
ax2.set_ylim([-DRANGE/10,DRANGE*1.1]) # 2,5,8
ax2.set_yticks(np.linspace(0, DRANGE, 5)) 
ax2.set_xlim([T0, Tvis])

# Kombinierte Legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=8)

# Energie-Achse nach vorne bringen
ax1.set_zorder(ax2.get_zorder() + 1)
ax1.patch.set_visible(False)

ax1.grid(True)
plt.tight_layout()
plt.savefig(f'EnergyX_{FNAME}.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# 2D plotting of energy
import numpy as np
import matplotlib.pyplot as plt

Nx=256
Ny=256
dd = 1e-4
EE  = np.zeros((Nx, Ny))
sgrid = np.linspace(0+dd,1-dd,Ny)
fgrid = np.linspace(0+dd,1-dd,Nx)

def Wf(φ):
    W1 = np.where(φ<0, 200*φ**2, 18*(φ*(1-φ))**2)
    W2 = np.where(φ>1, 200*(1-φ)**2, W1)
    return W2
  
def E(φ,s):
  f = 1-s
  if f<1:
    erg = s*np.log(s) + f*np.log(f) + β*φ*(s-s_sat) 
    erg += γ_l/ε * Wf(1-φ)
    erg += γ_c/ε * Wf(φ)

  else:
    erg = 0
  return erg

    

for i, φ in enumerate(fgrid):
    EE[i, :] = np.array([E(φ,s) for s in sgrid])

# create 2D plot
plt.pcolormesh(sgrid,fgrid, EE, shading="auto",cmap='gray')
plt.colorbar()
plt.contour(sgrid,fgrid, EE, levels=64, colors='white', linewidths=0.5)
plt.ylabel("$\\varphi_c$")
plt.xlabel("$s$")

In [ ]:
import matplotlib.pyplot as plt

def W(φ):
    W1 = np.where(φ<0, Λ*φ**2, 18*(φ*(1-φ))**2)
    W2 = np.where(φ>1, Λ*(1-φ)**2, W1)
    return W2

φ = np.linspace(-0.5,1.5,500)
Wφ = W(φ)
plt.figure(figsize=(4,3))
plt.plot(φ,18*(φ*(1-φ))**2,':',color='gray')
plt.plot(φ,Wφ)
plt.ylim([-0.1,3])
plt.xlim([-0.19,1.19])
plt.grid(True)
plt.xlabel('$\\varphi$')
plt.ylabel('$W(\\varphi)$')
plt.title('(a)', fontsize=10)
plt.savefig('Wfunction.pdf',dpi=300,bbox_inches='tight')

In [ ]:
# 2D plotting of energy
import numpy as np
import matplotlib.pyplot as plt

Nx=256
Ny=256
dd = 1e-4
EE  = np.zeros((Nx, Ny))
sgrid = np.linspace(0+dd,1-dd,Ny)
fgrid = np.linspace(0+dd,1-dd,Nx)

def Wf(φ):
    W1 = np.where(φ<0, Λ*φ**2, 18*(φ*(1-φ))**2)
    W2 = np.where(φ>1, Λ*(1-φ)**2, W1)
    return W2
  
def E(φ,s):
  f = 1-s
  if f<1:
    erg = s*np.log(s) + f*np.log(f) + β*φ*(s-s_sat) - β*(1-s_sat)
    erg += γ_l/ε * Wf(1-φ)
    erg += γ_c/ε * Wf(φ)
  else:
    erg = 0
  return erg

for i, φ in enumerate(fgrid):
    EE[i, :] = np.array([E(φ,s) for s in sgrid])

# Berechne den Gradienten
dE_dphi, dE_ds = np.gradient(EE, fgrid, sgrid)

# Erstelle ein gröberes Gitter für den Quiver-Plot
skip = 16  # Nur jeden 16. Pfeil anzeigen
S, F = np.meshgrid(sgrid, fgrid)

# create 2D plot
plt.figure(figsize=(4, 3))
plt.pcolormesh(sgrid, fgrid, EE, shading="auto", cmap='gray',vmin=0,vmax=10)
plt.colorbar()
plt.contour(sgrid, fgrid, EE, levels=32, colors='white', linewidths=0.5)

# Quiver-Plot (negativer Gradient zeigt Richtung des Abstiegs)
plt.quiver(S[::skip, ::skip], F[::skip, ::skip], 
           -dE_ds[::skip, ::skip], -dE_dphi[::skip, ::skip],
           color='k', alpha=0.7, scale=300)

plt.ylabel("$\\varphi_c=1-\\varphi_{\\ell}$")
plt.xlabel("$s$")
plt.title('(b)', fontsize=10)

plt.savefig('Energy_Landscape.png',dpi=300,bbox_inches='tight')